In [ ]:
!pip install -r requirements.txt
import api_functions as ap

In [ ]:
# Download kaggle data
path = kagglehub.dataset_download("ludmin/billboard")

In [ ]:
#Create a DataFrame
df_songs = pd.read_csv('hot100.csv')

In [ ]:
#Drop image url column
df_songs = df_songs.drop(columns=['Image URL'])

In [ ]:
#Drop duplicates
df_songs = df_songs.drop_duplicates()

In [ ]:
#Filter for only for top songs and only since 1990
df_songs = df_songs[(df_songs['Date'] >= '1990-01-01') & (df_songs['Peak Position'] = 1)]

In [ ]:
# Drop duplicates based on Artist and Song, keeping the first occurrence (best Peak Position)
df_songs = df_songs.drop_duplicates(subset=['Artist', 'Song'], keep='first')

In [ ]:
# Set variables

df = df_songs

load_dotenv()
api_key = os.getenv("api_key")

In [ ]:
# Process all songs
df = ap.add_genius_data(
    df,
    api_key,
    save_every=100)

# Save intermediate results
ap.save_results(df, "billboard_with_lyrics_intermediate.csv")

# Save final results
ap.save_results(df, "billboard_with_lyrics_final.csv")

In [ ]:
#Update missing lyrics

csv_path = "billboard_with_lyrics_final.csv"
ap.update_missing_lyrics(csv_path, api_key)

In [ ]:
#Clean lyrics
ap.update_and_clean_lyrics(csv_path, api_key, 250)

In [ ]:
#Remove not needed data
df = df[df['Peak Position'] == 1]
df = df.drop(columns=['genius_url'])
df = df.drop(columns=['match_score'])

In [ ]:
# Apply the function to add genres from Spotify
df['genre'] = df['Artist'].apply(ap.get_genre)

In [ ]:
#Extract the primary genre and add Primary genre column
df['Primary genre'] = df['genre'].tolist()
df['Primary genre'] = df['genre'].str.extract(r"\['(.*?)'")

In [ ]:
#Apply the function to get the final genre based on clustering of Primary genre values
df['Final_genre'] = ap.cluster_genres(df, 'Primary genre')

In [ ]:
#Capitalize column names and remove empty spaces
df.columns = df.columns.str.capitalize()
df.columns = df.columns.str.replace('_', '', regex=False)

In [ ]:
#Drop not needed columns 
df = df.drop(columns=['Rank', 'Last_week', 'Peak_position', 'Weeks_in_charts'])

In [ ]:
#Export the file
df.to_csv('billboard_with_lyrics_final_cleaned.csv', index=False)